In [32]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

from utilidades.validar import validar
from utilidades.graficar import ver_dispersion

import re 

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

mejorPruebaActual = pd.read_csv('submits/limpieza_descripciones_retuneo_LGBM_300d.csv')

In [3]:
#trainMejorado = pd.read_csv("data/train_limpio4_bis.csv")
#testMejorado = pd.read_csv("data/test_limpio4.csv")

In [4]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 71 columns):
tipodepropiedad_Apartamento                      240000 non-null int64
tipodepropiedad_Casa en condominio               240000 non-null int64
tipodepropiedad_Casa                             240000 non-null int64
tipodepropiedad_Terreno                          240000 non-null int64
tipodepropiedad_Terreno comercial                240000 non-null int64
tipodepropiedad_Local Comercial                  240000 non-null int64
tipodepropiedad_Quinta Vacacional                240000 non-null int64
tipodepropiedad_Oficina comercial                240000 non-null int64
tipodepropiedad_Edificio                         240000 non-null int64
tipodepropiedad_Casa uso de suelo                240000 non-null int64
tipodepropiedad_Local en centro comercial        240000 non-null int64
tipodepropiedad_Bodega comercial                 240000 non-null int64
tipodepropiedad_Otros                

In [5]:
mejorPruebaActual.head()

,id,target
0,4941,5.391570e+06
1,51775,1.037451e+06
2,115253,2.415119e+06
3,299321,1.222533e+06
4,173570,5.561620e+05


In [6]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

In [8]:
dfinicial.precio.max()

12525000

In [7]:
dfinicial[dfinicial['metrostotales']>225].precio.max()

12516000

In [13]:
dfinicial[dfinicial['metrostotales']>225].precio.mean()

3770771.685993544

In [14]:
dfinicial[dfinicial['metrostotales']<=225].precio.mean()

2155574.1087928787

# Se define la funcion

In [18]:
def generar_subida(DFTRAIN, metodo, DFTEST, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)    
    
    X, y = DFTRAIN.drop( columns = ['precio'] ), DFTRAIN["precio"]

    metodo.fit(X,y)
    
    subida = pd.DataFrame( metodo.predict(DFTEST) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    #Este modelo no necesita las euristicas,pero igual no da muy bien
    
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

#### Evalúo los parametros iniciales

In [8]:
GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [16]:
#dfinicial1 = dfinicial[dfinicial['metrostotales']<225].drop( columns = ['dia'] )
#dftest1 = dftest.drop( columns = ['dia'] )


#dfinicial1  = trainMejorado[trainMejorado['metrostotales']<225].drop( columns = ['dia'] )
#dftest1 = testMejorado.drop( columns = ['dia'] )


#dfinicial1 = dfinicial2.drop( columns = [ 'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos',  'escuelascercanas'
#                           ])                                        
 
#dftest1 = dftest2.drop( columns = [  'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos', 'escuelascercanas'
#                           ])   

reg = GradientBoostingRegressor(
)



X, y, subida = generar_subida( dfinicial, reg, dftest, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO 
#(No es la mejor forma pero sirve como aproximacion si el Kfold tarda mucho)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

540277.0

### Verifico formalmente el metodo con Kfold

In [18]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor, False)
#da error de 795000

Error de fold N°0 es : 796954.7472719415
Error de fold N°1 es : 797055.6763865778
Error de fold N°2 es : 798668.9923507419
Error de fold N°3 es : 796861.2225620802
Error de fold N°4 es : 790163.0381804889


# Modifico los hiperparametros propios del Gradient Boosting para maximizar su eficacia

In [22]:

gradientBoostingOptimizado = GradientBoostingRegressor(
    alpha=0.9, #no cambia nada
    max_depth=4,
    n_estimators=100, #el default es 100 asique deberia dar bastante
    learning_rate=0.5,
    min_samples_leaf=250,
    subsample=0.5,
    validation_fraction=0.1
)

    
X, y, subida = generar_subida( dfinicial, gradientBoostingOptimizado, dftest, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )    

483449.0

In [21]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)
#su error ronda los 707000

Error de fold N°0 es : 715277.06581662
Error de fold N°1 es : 705703.5965765929
Error de fold N°2 es : 709066.5240343931
Error de fold N°3 es : 706036.579450608
Error de fold N°4 es : 704949.2174846162


### Con 150 estimadores de n me podria dar mejor

In [ ]:
gradientBoostingOptimizado = GradientBoostingRegressor(
    alpha=0.9, #no cambia nada
    max_depth=4,
    n_estimators=150, #el default es 100 asique deberia dar bastante
    learning_rate=0.5,
    min_samples_leaf=250,
    subsample=0.5,
    validation_fraction=0.1
)

    
X, y, subida = generar_subida( dfinicial, gradientBoostingOptimizado, dftest, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )   

In [21]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=150,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)
#Con 150 estimadores de n corre un poco mejor

NameError: name 'X' is not defined

# Trabajo con features preexistentes

## Quito las propiedades muy grandes porque perjudican el calculo

### Esto se debe a que el algoritmo predice mejor las viviendas de menor costo,y las que suelen tener un mayor precio son las mas grandes

In [15]:
dfinicial[dfinicial['metrostotales']>225].precio.mean()

3770771.685993544

In [16]:
dfinicial[dfinicial['metrostotales']<=225].precio.mean()

2155574.1087928787

In [23]:
dfinicial1 = dfinicial[dfinicial['metrostotales']<225]#.drop( columns = ['dia'] )
dftest1 = dftest#.drop( columns = ['dia'] )
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )    

488765.0

In [24]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 588995.4266792418
Error de fold N°1 es : 586227.4738019543
Error de fold N°2 es : 591839.4391255038
Error de fold N°3 es : 585896.726326693
Error de fold N°4 es : 589426.8528888001


### su error ronda los 588000 no obstante primero subio y el segundo bajo porque el primero igual calcula las propiedades grandes y el segundo no,asique esto prueba que las propiedades grandes enturbiecen el resultado aunque no lo pueda usar

In [20]:

gradientBoostingOptimizado = GradientBoostingRegressor(
    alpha=0.9, #no cambia nada
    max_depth=4,
    n_estimators=100, #el default es 100 asique deberia dar bastante
    learning_rate=0.5,
    min_samples_leaf=250,
    subsample=0.5,
    validation_fraction=0.1
)

## Le quito features innecesarios como día

In [ ]:
dfinicial1 = dfinicial.drop( columns = ['dia'] )
dftest1 = dftest.drop( columns = ['dia'] )
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )  

In [27]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 714109.9543325376
Error de fold N°1 es : 704573.8540342319
Error de fold N°2 es : 713927.6567804025
Error de fold N°3 es : 713185.0902547204
Error de fold N°4 es : 704212.5961690441


In [28]:
# Su error ronda los 710000
#sacar el dia no afectó nada,probablemente el 
#mismo algoritmo al determinar que no aporta 
#informacion decide obviarlo cuando está,por 
#lo que no cambia nada si está o no

## despues quitar otros features pero bueno

# Creacion de features propios

### Por lo previamente establecido,los precios de viviendas grandes son mas dificiles de calcular,por lo que me agrego otros features para las mmismas

In [26]:
dfinicial1 = dfinicial
dfinicial1['viviendaGrande'] = (dfinicial['metrostotales']>225).astype(int)
dfinicial1.head()

,tipodepropiedad_Apartamento,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Local Comercial,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Oficina comercial,tipodepropiedad_Edificio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Local en centro comercial,...,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,dia,mes,anio,viviendaGrande
0,1,0,0,0,0,0,0,0,0,0,0,...,80,0,0,0,0,0,2273000,23,8,2015,0
1,0,1,0,0,0,0,0,0,0,0,0,...,180,0,0,0,1,1,3600000,28,6,2013,0
2,0,0,1,0,0,0,0,0,0,0,0,...,166,0,0,0,0,0,1200000,17,10,2015,0
3,0,0,1,0,0,0,0,0,0,0,0,...,67,0,0,0,1,1,650000,9,3,2012,0
4,1,0,0,0,0,0,0,0,0,0,0,...,95,0,0,0,0,0,1150000,7,6,2016,0


In [27]:
dfinicial1 = dfinicial
dfinicial1['viviendaGrande'] = (dfinicial['metrostotales']>225).astype(int)

dftest1 = dftest
dftest1['viviendaGrande'] = (dftest['metrostotales']>225).astype(int)
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )   

480494.0

In [28]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 713115.9393285706
Error de fold N°1 es : 701881.5620320033
Error de fold N°2 es : 715378.7555249189
Error de fold N°3 es : 709500.5813459036
Error de fold N°4 es : 706838.0468517158


### Apenas obtuve una "mejora",pruebo cambiando el criterio para considerar casas grandes

In [19]:
dfinicial1 = dfinicial
dfinicial1['viviendaGrande'] = (dfinicial['metrostotales']>250).astype(int)

dftest1 = dftest
dftest1['viviendaGrande'] = (dftest['metrostotales']>250).astype(int)
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )   

NameError: name 'gradientBoostingOptimizado' is not defined

In [36]:
#Aunque me dé mas de 585000, si me da menos de 707000 ya es una mejora, porque
#obviamente no me va a dar mejor que sin tener en cuenta las propiedades grandes,
#pero así al menos las estoy tratando de forma diferente

In [31]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 715538.5467012791
Error de fold N°1 es : 707510.2960080666
Error de fold N°2 es : 711790.2795080957
Error de fold N°3 es : 710487.8447637347
Error de fold N°4 es : 708807.7116772638


In [37]:
#Todavia no me dió bien,cambio devuelta el criterio

In [44]:
dfinicial1 = dfinicial
dfinicial1['viviendaGrande'] = ((dfinicial['metrostotales']>250) & (dfinicial['garages'])>1).astype(int)

dftest1 = dftest
dftest1['viviendaGrande'] = ((dftest['metrostotales']>250) & (dftest['garages'])>1).astype(int)
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )  

482620.0

In [45]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 720073.7221792672
Error de fold N°1 es : 708646.2239435158
Error de fold N°2 es : 710286.9572230885
Error de fold N°3 es : 707838.0755648133
Error de fold N°4 es : 707547.304656751


## En conclusion,esto no sirve,no aporta ninguna mejora significativa a la prediccion sino que,al contrario,lo empeora

# Pruebo con features del df original

### Me creo una funcion que determine si hay palabras "positivas" respecto a la vivienda en la descripcion de cada una en el df original

In [23]:
train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [57]:
prueba_descripcion=train.iloc[2]['descripcion']
print(prueba_descripcion)

descripcion 
la mejor ubicacion residencial en el oriente de la ciudad urbi versalles, te invito a compartir una nueva forma de vida en la zona urbana de guadalajara en el mejor desarrollo residencial en su tipo, ven y descubre un concepto residencial integral unico: 
 por su amplitud y tranquilidad. 
 por su seguridad y confort. 
 por su diseños. 
 por sus bellos terminados. 
 el mejor precio del mercado. 
 por su exelente ubicación, estamos cerca de todo y de todos! 
 vias de comunicacions rapidas y modernas. 
 centros comerciales dentro del fraccionamiento . 
 bello ingreso privado y vigilado. 
 acceso controlado en cada coto, con guardias y vigilancia por video. 
 perimetro totalmente cerrado. 
 cableado oculto. 
 amplios jardines y areas verdes. 
 areas infantiles y deportivos. 
 escuelas, centros comerciales, hospitales, farmacias todo lo que necesitas a la mano. 
 tranquilidad garantizada. 
 3 recamaras, amplia sala, cocina integral , 2 1/2 baños, totalmente ind

In [59]:
prueba_descripcion_2=train.iloc[1]['descripcion']
print(prueba_descripcion_2)

<p>entre sonora y guerrero, atr&aacute;s del hospital &aacute;ngeles del pedregal, en excelente estado de conservaci&oacute;n, construida en 3 niveles, estacionamiento para 2 autos, 3 rec&aacute;maras con pisos de madera natural la principal con vestidor y ba&ntilde;o, 2 ba&ntilde;os completos pisos y muros de loseta espa&ntilde;ola, canceler&iacute;a de aluminio, 2 medios ba&ntilde;os, cocina integral con piso de loseta espa&ntilde;ola, sala de t.v., sala con piso de loseta espa&ntilde;ola, vest&iacute;bulo, comedor con piso de madera, l&iacute;nea telef&oacute;nica, terraza techada con policarbonato puede ser antecomedor, cuarto de servicio con piso de loseta, fachada con policarbonato..&nbsp; 4 casas</p>


In [45]:
train['descripcionPositiva'] =(train['descripcion'].str.find('bonito'))

In [46]:
train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,descripcionPositiva
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0,-1.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0,-1.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0,1177.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0,-1.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0,0.0


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(y_test, pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel("Valores reales", fontsize=16)
ax.set_ylabel("Valores predichos", fontsize=16);

In [ ]:
#reg = LinearRegression(normalize=True, n_jobs = -1)
#reg.fit(X,y)

In [ ]:
train.info()

In [ ]:
train['descripcion'] = train['descripcion'].to_string()
train.info()

In [ ]:
train.head()